In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from source.TrainDataGeneration import append_ORD_TM,append_ORD_VOL,append_TM_GP,GetGroupDataFrame,append_STEP5

In [5]:
new_index

MultiIndex([(20170901, 'HK0000295359',  0.0),
            (20170901, 'HK0000295359',  1.0),
            (20170901, 'HK0000295359',  2.0),
            (20170901, 'HK0000295359',  3.0),
            (20170901, 'HK0000295359',  4.0),
            (20170901, 'HK0000295359',  5.0),
            (20170901, 'HK0000295359',  6.0),
            (20170901, 'HK0000295359',  7.0),
            (20170901, 'HK0000295359',  8.0),
            (20170901, 'HK0000295359',  9.0),
            ...
            (20170929, 'USU652221081', 29.0),
            (20170929, 'USU652221081', 30.0),
            (20170929, 'USU652221081', 31.0),
            (20170929, 'USU652221081', 32.0),
            (20170929, 'USU652221081', 33.0),
            (20170929, 'USU652221081', 34.0),
            (20170929, 'USU652221081', 35.0),
            (20170929, 'USU652221081', 36.0),
            (20170929, 'USU652221081', 37.0),
            (20170929, 'USU652221081', 38.0)],
           names=['ORD_DD', 'ISU_CD', 'TM_GP'], length=81900)

In [4]:
TGDF

Unnamed: 0  MBR_NO  BRN_NO  ASKBID_TP_CD  \
ORD_DD   ISU_CD       TM_GP                                             
20170901 HK0000295359 6.0             0       1       1             1   
                      7.0             1       1       1             1   
                      32.0            2       1       7             2   
                      35.0            3       1       7             2   
                      36.0            4       1       7             2   
...                                 ...     ...     ...           ...   
20170929 USU652221081 17.0      5849428      86     300             1   
                      18.0      5849429      86     300             2   
                      23.0      5849430      86     300             2   
                      34.0      5849431      86     300             2   
                      36.0      5849432      86     300             2   

                             MODCANCL_TP_CD  ORD_QTY   ORD_VOL  ORD_ACPT_TM  
ORD_DD   ISU_CD       TM_GP                                                  
20170901 HK0000295359 6.0                 1    16000  33060000          NaN  
                      7.0                 1    14000  28920000          NaN  
                      32.0                1       50    101750          NaN  
                      35.0                1      100    203500          NaN  
                      36.0                1       50    102000          NaN  
...                                     ...      ...       ...          ...  
20170929 USU652221081 17.0                3     6819   9955740          NaN  
                      18.0                1        7     10080          NaN  
                      23.0                1     2000   2880000          NaN  
                      34.0                2     2000   2900000          NaN  
                      36.0                2     2000   2920000          NaN  

[13716822 rows x 8 columns]

In [3]:
TGDF.reindex(new_index)

ValueError: cannot handle a non-unique multi-index!

In [2]:
TickDataname='GDF2022_5_4_8_25.csv'
TickGDF=pd.read_csv('CollectedData/'+TickDataname)
Dataname='norm6featNewPrc'

TGDF=TickGDF[(TickGDF['TM_GP']>=0)&(TickGDF['TM_GP']<39)]
TGDF=TGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])

new_index = pd.MultiIndex.from_product(TGDF.index.levels)
new_df = TGDF.reindex(new_index)
new_df = new_df.ffill()
new_df=new_df.reset_index()
Train_df=new_df
Train_df.loc[new_df['TM_GP']==0,'시가']=Train_df.loc[new_df['TM_GP']==1,'시가'].values

ValueError: cannot handle a non-unique multi-index!

In [ ]:
Train_df['10단계호가합계잔량']=Train_df['매수10단계호가합계잔량']+Train_df['매도10단계호가합계잔량']

In [ ]:
qtylist=['ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_RQTY', 
       'ASK_STEP3_BSTORD_RQTY',  'ASK_STEP4_BSTORD_RQTY', 
       'ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_RQTY', 
       'BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_RQTY', 
       'BID_STEP4_BSTORD_RQTY','BID_STEP5_BSTORD_RQTY',
        '매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량']

Train_df[qtylist]=Train_df[qtylist].div(Train_df['10단계호가합계잔량'],axis=0).values

Train_df['10단계호가합계잔량']=np.log(new_df['10단계호가합계잔량'])

In [ ]:
divdict={}
# divdict['10단계호가합계잔량']=qtylist
divdict['시가']=['고가','저가','직전체결가격']
loglist=[]
loglist+=['고가','저가','직전체결가격','시가']
# loglist+=['고가','저가','직전체결가격','시가']
# loglist+=['10단계호가합계잔량']
minmaxnormlist=[]
minmaxnormlist+=['매도총호가잔량', '매수총호가잔량', '고가', '저가', 
           '직전체결가격']

In [ ]:
for divcol in divdict:
    Train_df[divdict[divcol]]=Train_df[divdict[divcol]].div(Train_df[divcol],axis=0).values

In [ ]:
Train_df[loglist]=np.log(Train_df[loglist])

In [ ]:
Train_df[minmaxnormlist]=(Train_df[minmaxnormlist]-Train_df[minmaxnormlist].min())/(Train_df[minmaxnormlist].max()-Train_df[minmaxnormlist].min())

In [ ]:
ORDTickDataname='GDF2022_4_12_19_51.csv'
ORDTickGDF=pd.read_csv('CollectedData/'+ORDTickDataname)

In [ ]:
ORDTickGDF

In [ ]:
ORDTickGDF.columns

In [ ]:
ORDTickGDF['MODCANCL_TP_CD']

In [ ]:
ORDTickGDF['ASKBID_TP_CD']

In [ ]:
OTGDF=ORDTickGDF[(ORDTickGDF['TM_GP']>=0)&(ORDTickGDF['TM_GP']<39)]
OTGDF=OTGDF[OTGDF['MODCANCL_TP_CD']!=2]

# OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)*(2-OTGDF['MODCANCL_TP_CD'])
# OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)#'norm6featNew'
OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)*OTGDF['ORD_PRC']#'norm6featNewPrc'
# OTGDF=OTGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])

In [ ]:
groupcolumns=['ORD_DD', 'ISU_CD', 'MBR_NO', 'BRN_NO','TM_GP']
sumcolumns=['NET_ORD_QTY']
meancolumns,lastcolumns=[],[]

In [ ]:
OTGDF=GetGroupDataFrame(OTGDF,groupcolumns,sumcolumns,meancolumns,lastcolumns)

In [ ]:
feat_cols=['ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_RQTY', 
       'ASK_STEP3_BSTORD_RQTY',  'ASK_STEP4_BSTORD_RQTY', 
       'ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_RQTY', 
       'BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_RQTY', 
       'BID_STEP4_BSTORD_RQTY','BID_STEP5_BSTORD_RQTY',
        '매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',
           '시가', '직전체결가격','NET_ORD_QTY']

In [ ]:
feat_cols=['매도총호가잔량', '매수총호가잔량', '고가', '저가', 
           '직전체결가격','NET_ORD_QTY']

feat_cols=['매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',
           '시가', '직전체결가격','NET_ORD_QTY']

In [ ]:
mbrnlist1=[(5,194),(2,155),(12,100),(17,29),(42,1),(44,1),(50,92),(2,83),(4,10118),(8,298)]
mbrnlist2=[(5,194),(12,100),(2,155),(17,29),(42,1),(44,1),(2,83),(4,10118),(4,9997),(50,91)]
mbrnlist=mbrnlist1+[a for a in mbrnlist2 if a not in mbrnlist1]

In [ ]:
Train_df[feat_cols[:-1]]

In [ ]:
for mbr,brn in mbrnlist:
    print(mbr,brn)
    OTGDF_temp=OTGDF[(OTGDF['MBR_NO']==mbr) &(OTGDF['BRN_NO']==brn)]
    OTGDF_temp=OTGDF_temp.set_index(['ORD_DD','ISU_CD','TM_GP'])
    OTGDF_temp=OTGDF_temp.reindex(new_index)
    OTGDF_temp['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].fillna(0)
    Train_df['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].values
    Train_df['ylabel']=Train_df['NET_ORD_QTY']
    Train_df['ylabel'][:-1]=(2*(Train_df['NET_ORD_QTY']>0)+1*(Train_df['NET_ORD_QTY']==0))[1:]
    Train_df['ylabel'][38::39]=1
    print(Train_df['NET_ORD_QTY'].sum())    

    SaveDirPath='/Data/Min/TransLOB/TrainData/'
    now=datetime.now()
    DataSubfix=str(mbr)+'_'+str(brn)+str(now.year)+'_'+str(now.month)+'_'+str(now.day)
    DataSubfix=str(mbr)+'_'+str(brn)+Dataname
    TrainDataname='Train_ORD'+'_'+DataSubfix
    TrainLabelDataname='Train_ORD_Label_'+'_'+DataSubfix
    np.save(SaveDirPath+TrainDataname,Train_df[feat_cols].values)
    np.save(SaveDirPath+TrainLabelDataname,Train_df['ylabel'])
    print("path:",SaveDirPath+TrainDataname)
    print("path:",SaveDirPath+TrainLabelDataname)
with open(SaveDirPath+'DataInformationORD.txt', "a+") as f:
    f.write("TrainDataname:"+TrainDataname+'\n')
    f.write("TrainLabelDataname:"+TrainLabelDataname+'\n')
    f.write("TickDataname:"+TickDataname+'\n')
    f.write("ORDTickDataname:"+ORDTickDataname+'\n')

    f.write("divdict:"+'\n')
    for col,llist in divdict.items():
        f.write(str(col)+'\t'+str(llist))
    f.write('\n')
    f.write("loglist:"+'\n')
    for col in loglist:
        f.write(str(col)+'\t')
    f.write('\n')

    f.write("feat_cols:"+'\n')
    for col in feat_cols:
        f.write(str(col)+'\t')
    f.write('\n')


In [ ]:
Train_df[feat_cols].values

In [ ]:
Train_df[feat_cols].values.shape

In [ ]:
Train_df['ylabel'].values

a

SaveDirPath='/Data/Min/TransLOB/TrainData/'
now=datetime.now()
TrainDataname='Train_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
TrainLabelDataname='Train_Label_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
np.save(SaveDirPath+TrainDataname,Train_df[feat_cols].values)
np.save(SaveDirPath+TrainLabelDataname,increment)
with open(SaveDirPath+'DataInformation.txt', "a+") as f:
    f.write("TrainDataname:"+TrainDataname+'\n')
    f.write("TrainLabelDataname:"+TrainLabelDataname+'\n')
    f.write("TickDataname:"+TickDataname)
    for col in feat_cols:
        f.write(str(col)+'\t')
    f.write('\n')
print("path:",SaveDirPath+TrainDataname)
print("path:",SaveDirPath+TrainLabelDataname)